# Equal-Weight S&P 500 Index Fund

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [23]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter 
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course.

There's a static version of the S&P 500 constituents available here. Click this link to download them now. Move this file into the starter-files folder so it can be accessed by other files in that directory.

In [24]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a secrets.py file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

Move the file into the same directory as this Jupyter Notebook before proceeding.

In [25]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
Now it's time to structure our API calls to IEX cloud.

We need the following information from the API:

Market capitalization for each stock

Price of each stock

In [26]:
symbol = 'AAPL'                                              #add appendage at the end of url that passes in API token to get permission of access       
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'        #use f-string to stich a variable into a url
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 94368359, 'calculationPrice': 'close', 'change': 2.96, 'changePercent': 0.02015, 'close': 155, 'closeSource': 'lfiiaofc', 'closeTime': 1708988704573, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 152.332, 'delayedPriceTime': 1734421913054, 'extendedChange': -0.29, 'extendedChangePercent': -0.00196, 'extendedPrice': 153.01, 'extendedPriceTime': 1711307352713, 'high': 155.54, 'highSource': 'eni1 lety ddriame pcu5e', 'highTime': 1682364674455, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 154.16, 'iexCloseTime': 1748829116533, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 146.06, 'iexOpenTime': 1732067100559, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1749001472182, 'latestPrice': 152.7, 'latestSource': 'Close', 'latestTime': 'November 11, 2022', 'latestUpdate': 1746882489830, 'latestVolume': 95554706, 'low': 150.64, 'lowSource': 'ec ruln1d eeytidp

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy.

With that said, the data isn't in a proper format yet. We need to parse it first.

In [27]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price,market_cap/1e12) #divide the market cap by a trillion

152.7 2.443653601559


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [28]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [29]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,152.7,2443653601559,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a for loop.

In [30]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns
        ),
    ignore_index=True 
    ) #calling it manually from API for huge amount of data is a very slow process, hence we need to use a batch API calls in next section

In [31]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,149.03,45216795910,N/A
1,AAL,15.57,9785166874,N/A
2,AAP,186.49,11243764220,N/A
3,AAPL,156.70,2417215682489,N/A
4,ABBV,156.36,278518195567,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

 

In [32]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [33]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,149.99,45014133701,N/A
1,AAL,15.58,9892179367,N/A
2,AAP,192.57,11299013964,N/A
3,AAPL,152.40,2391912501586,N/A
4,ABBV,155.88,277119345887,N/A
...,...,...,...,...
496,YUM,126.96,35130614242,N/A
497,ZBH,116.02,24860198070,N/A
498,ZBRA,264.87,14008444408,N/A
499,ZION,56.37,8566936657,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [34]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print('that is not a number!')
    print('Please try again')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio: 1000000


In [35]:
position_size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/ final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,149.99,45014133701,13
1,AAL,15.58,9892179367,128
2,AAP,192.57,11299013964,10
3,AAPL,152.40,2391912501586,13
4,ABBV,155.88,277119345887,12
...,...,...,...,...
496,YUM,126.96,35130614242,15
497,ZBH,116.02,24860198070,17
498,ZBRA,264.87,14008444408,7
499,ZION,56.37,8566936657,35


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [36]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our .xlsx File

Formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

String format for tickers

$XX.XX format for stock prices

$XX,XXX format for market capitalization

Integer format for the number of shares to purchase

In [37]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,       
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,       
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,       
        'border' : 1
    }
)

### Applying the Formats to the Columns of Our .xlsx File

We can use the set_column method applied to the writer.sheets['Recommended Trades'] object to apply formats to specific columns of our spreadsheets.

Here's an example:

writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B

18, #This tells the method to apply a column width of 18 pixels
                     
string_template #This applies the format 'string_template' to the column)

In [38]:
#writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format) #these doesn't affect the column header formatting
#writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
#writer.save()

#writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format) #these are all column formatting for column header
#writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
#writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
#writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself".

Let's simplify this by putting it in 2 loops:

In [39]:
column_format = {
    'A' : ['Ticker', string_format],
    'B' : ['Stock_Price', dollar_format],
    'C' : ['Market Capitalization', dollar_format],
    'D' : ['Number of Shares to Buy', integer_format]
}

for column in column_format.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_format[column][1]) #formatting everything below header
    writer.sheets['Recommended Trades'].write(f'{column}1', column_format[column][0], column_format[column][1]) #formatting column header

### Saving Our Excel Output

Saving our Excel file is very easy:

In [40]:
writer.save()